In [7]:
@time include("setup.jl")

Total number of nodes on fault = 121
Average node spacing = 0.0004
dt = 0.00007477186690791
Setup Complete
  0.902617 seconds (494.30 k allocations: 28.702 MiB, 0.79% gc time)


In [ ]:
# Plot friction parameters
using Gadfly

fricPlot = plot(layer(x = (cca-ccb).*1e3 + 10, y = FltX,Theme(default_color=colorant"red"), Geom.point),
                layer(x = Seff/1e6, y = FltX, Theme(default_color=colorant"deepskyblue"), Geom.point),
                layer(x = tauo/1e6, y = FltX, Theme(default_color=colorant"green"), Geom.point),
                Guide.xlabel("Scaled (a-b)/ Stress value"),
                Guide.ylabel("Depth (m)"),
                Guide.title("Rate and state friction/Stress"),
                Coord.Cartesian(ymin=-24))

In [13]:
@time include("timesolver.jl")

  1.105180 seconds (2.19 M allocations: 106.681 MiB, 1.35% gc time)


In [16]:
tau

121×1 Array{Float64,2}:
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     0.0  
     ⋮    
  -439.889
  -439.585
  -448.175
  -459.314
  -473.52 
  -574.098
  -679.787
  -144.314
  -539.387
  -109.667
 -9921.3  
 57244.0  

In [ ]:
ddd = Inf 

if ddd == Inf
    error("Hello World!")
end

In [ ]:
j = 1
IDstate = 3

In [ ]:
            if IDstate == 1
                psi2[j] = psi[j] + 0.5*dt*( (Vo[j]./xLf[j]).*exp(-psi[j]) 
                                            - abs(Vf[j])./xLf[j] 
                                            + (Vo[j]./xLf[j]).*exp(-psi1[j]) 
                                            - abs(Vf1[j])./xLf[j] )

            elseif IDstate == 2
                VdtL = 0.5*abs(Vf1[j] + Vf[j])*dt/xLf[j]
                if VdtL < 1e-6
                    psi2[j] = log( exp(psi[j]-VdtL) + Vo[j]*dt/xLf[j] -
                                  0.5*Vo[j]*0.5*abs(Vf1[j] + Vf[j])*dt*dt/(xLf[j]^2))
                else
                    psi2[j] = log(exp(psi[j]-VdtL) + (Vo[j]/abs(Vf[j]))*(1-exp(-VdtL)))
                end

            elseif IDstate == 3
                psi1[j] = VdtL

                if ~any(imag(psi1)) == 0
                    return
                end
            end


In [ ]:
VdtL